In [1]:
import numpy as np
import pandas as pd
import json
import urllib.request
import mwclient


matchup_stats = pd.read_csv('matchup_stats.csv').reset_index(drop=True)
game_count = matchup_stats.groupby(by=['champion_name', 'enemy_champion'], as_index=False).size()
matchup_stats = matchup_stats.groupby(by=['champion_name', 'enemy_champion'], as_index=False).mean()
matchup_stats['game_count'] = game_count['size']
matchup_stats = matchup_stats[matchup_stats['game_count']>5]
matchup_stats = matchup_stats.sort_values(by='game_count').rename(columns={'gold_diff@8': 'm_gd@8', 'cs_diff@8': 'm_cd@8',
                                                                    'xp_diff@8': 'm_xpd@8', 'cs_diff@14': 'm_cd@14',
                                                                    'xp_diff@14': 'm_xpd@14', 'cs_diff@14': 'm_cd@14',
                                                                    'kills_assists@8': 'm_ka@8', 'kills_assists@14': 'm_ka@14',
                                                                    'gold_diff@14': 'm_gd@14', 'jungle_prox': 'm_jp',
                                                                    'jungle_prox_diff': 'm_jpd'})
matchup_stats

,champion_name,enemy_champion,m_gd@8,m_cd@8,m_xpd@8,m_ka@8,m_gd@14,m_cd@14,m_xpd@14,m_ka@14,m_jp,m_jpd,game_count
300,Camille,Darius,-97.500000,-5.833333,326.666667,0.500000,-425.333333,-11.833333,260.333333,1.166667,0.100092,-0.016181,6
396,Corki,Zoe,48.000000,-0.833333,-92.000000,0.500000,-217.666667,-5.833333,-578.500000,0.833333,0.197180,-0.038835,6
393,Corki,Yasuo,7.333333,-2.000000,-33.333333,0.000000,333.666667,5.000000,36.000000,0.500000,0.192325,-0.022423,6
1793,Sion,Olaf,-431.000000,-7.000000,111.333333,0.000000,-800.166667,-17.833333,-84.833333,0.333333,0.029820,-0.049006,6
736,Gwen,Lillia,-346.833333,-12.000000,-349.166667,0.333333,-640.666667,-15.333333,-468.166667,0.500000,0.041609,-0.027739,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...
253,Azir,Sylas,56.354839,5.266129,123.637097,0.290323,52.467742,7.782258,187.774194,1.129032,0.189945,-0.020597,124
1260,Lucian,Zeri,128.261780,1.256545,46.209424,0.523560,157.570681,1.141361,80.392670,1.361257,0.141031,0.002470,191
2489,Zeri,Lucian,-128.261780,-1.256545,-46.209424,0.345550,-157.570681,-1.141361,-80.392670,1.293194,0.138561,-0.002470,191
2502,Zeri,Sivir,23.409483,1.995690,25.310345,0.275862,87.392241,2.659483,20.711207,1.168103,0.108509,-0.010559,232


### ADDING MODEL STATS TO MAIN STATS

In [2]:
from sqlalchemy import create_engine
import base64

def del_team_tag(player_name):
    if len(player_name.split(" ", 1)) == 1:
        return str(player_name)
    else:
        return str(player_name.split(" ", 1)[1])

pd.options.mode.chained_assignment = None 
pg_password = 'U3ViYXJ1MjE2NQ=='
decrypted_pg_pwd = base64.b64decode(pg_password).decode('utf-8')
engine_get = create_engine('postgresql://postgres:%s@localhost:5432/OFFICIAL_GAMES' % decrypted_pg_pwd)

mid_stats = pd.read_sql("select player_stats.*, league_info.name as league_name from player_stats \
                        inner join league_info on player_stats.league_id=league_info.id where role in ('Top','Mid','Bot')", engine_get)
games = mid_stats['live_data_match_urn'].unique()

roles = ['Top', 'Mid', 'Bot']
fixed_mid_stats = pd.DataFrame()
for game in games:
    temp_stats = mid_stats[mid_stats['live_data_match_urn'] == game]
    
    for role in roles:
        temp_mid_stats = temp_stats[temp_stats['role'] == role]
        if len(temp_mid_stats.index) > 1:
            enemy_champ_1 = temp_mid_stats['champion_name'].iloc[1]
            enemy_champ_2 = temp_mid_stats['champion_name'].iloc[0]

            temp_mid_stats_1 = temp_mid_stats.iloc[[0]]
            temp_mid_stats_2 = temp_mid_stats.iloc[[1]]

            temp_mid_stats_1['enemy_champion'] = enemy_champ_1
            temp_mid_stats_2['enemy_champion'] = enemy_champ_2
            
            temp_mid_stats_1['player_name'] = del_team_tag(temp_mid_stats['summoner_name'].iloc[0])
            temp_mid_stats_2['player_name'] = del_team_tag(temp_mid_stats['summoner_name'].iloc[1])

            temp_mid_stats_1 = pd.merge(temp_mid_stats_1, matchup_stats,  how='left', 
                                        left_on=['champion_name','enemy_champion'], 
                                        right_on = ['champion_name','enemy_champion'])
            temp_mid_stats_2 = pd.merge(temp_mid_stats_2, matchup_stats,  how='left', 
                                left_on=['champion_name','enemy_champion'], 
                                right_on = ['champion_name','enemy_champion'])

            fixed_mid_stats = pd.concat([fixed_mid_stats, temp_mid_stats_1, temp_mid_stats_2])

fixed_mid_stats


,live_data_player_urn,side,date,summoner_name,patch,champion_name,role,first_blood_killer,result,game_time,...,m_cd@8,m_xpd@8,m_ka@8,m_gd@14,m_cd@14,m_xpd@14,m_ka@14,m_jp,m_jpd,game_count
0,live:lol:riot:player:f8ecff7b-1859-3884-919c-a...,blue,2022-05-31T14:55:00Z,RGO Sinmivak,12.09,Gwen,Top,0,1,25.378000,...,0.291667,-19.312500,0.291667,-261.750000,1.729167,-21.625000,0.916667,0.086287,0.013120,48.0
0,live:lol:riot:player:bead843c-5b38-3466-a6d7-0...,red,2022-05-31T14:55:00Z,Z10 mumus100,12.09,Gangplank,Top,0,0,25.378000,...,-0.291667,19.312500,0.479167,261.750000,-1.729167,21.625000,1.541667,0.073167,-0.013120,48.0
0,live:lol:riot:player:be07731d-87bc-390c-819f-4...,blue,2022-05-31T14:55:00Z,RGO Nite,12.09,Viktor,Mid,0,1,25.378000,...,1.800000,58.000000,0.500000,14.966667,0.866667,6.666667,1.166667,0.206108,-0.021730,30.0
0,live:lol:riot:player:012972c6-1d0d-3feb-b904-c...,red,2022-05-31T14:55:00Z,Z10 Sebekx,12.09,Ahri,Mid,0,0,25.378000,...,-1.800000,-58.000000,0.366667,-14.966667,-0.866667,-6.666667,1.300000,0.227838,0.021730,30.0
0,live:lol:riot:player:1a8706ab-e4d7-39cb-91fd-d...,blue,2022-05-31T14:55:00Z,RGO Lucker,12.09,Kai'Sa,Bot,0,1,25.378000,...,-0.782609,52.173913,0.869565,-230.260870,-2.565217,-66.173913,1.434783,0.116268,-0.044624,23.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,live:lol:riot:player:dbc379c3-4df2-370d-9047-0...,red,2023-02-28T11:17:00Z,DK Thanatos,13.04,Renekton,Top,0,0,36.026867,...,8.000000,55.000000,0.333333,614.666667,12.583333,165.000000,0.958333,0.075245,0.013812,24.0
0,live:lol:riot:player:eb5f065c-d4e7-3405-96f2-b...,blue,2023-02-28T11:17:00Z,BRO Ivory,13.04,Syndra,Mid,0,1,36.026867,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,live:lol:riot:player:6f4016f0-2e15-3fde-8d9b-7...,red,2023-02-28T11:17:00Z,DK Pullbae,13.04,Kassadin,Mid,0,0,36.026867,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,live:lol:riot:player:fbe1e871-a333-3eb9-a454-2...,blue,2023-02-28T11:17:00Z,BRO Enosh,13.04,Zeri,Bot,1,1,36.026867,...,-3.088235,-46.205882,0.735294,-508.441176,-4.441176,-221.588235,1.970588,0.147916,-0.007834,34.0


### CALCULATING ADJUSTED DIFFS

In [3]:
adjusted_mid_stats = pd.DataFrame()
for row in fixed_mid_stats.to_dict(orient='records'):
    a_gd_8 = row['gold_diff@8'] - row['m_gd@8']
    a_cd_8 = row['cs_diff@8'] - row['m_cd@8']
    a_xpd_8 = row['xp_diff@8'] - row['m_xpd@8']
    a_ka_8 = row['kills_assists@8'] - row['m_ka@8']
    
    a_gd_14 = row['gold_diff@14'] - row['m_gd@14']
    a_cd_14 = row['cs_diff@14'] - row['m_cd@14']
    a_xpd_14 = row['xp_diff@14'] - row['m_xpd@14']
    a_ka_14 = row['kills_assists@14'] - row['m_ka@14']
    
    a_jp = row['jungle_prox'] - row['m_jp']
    a_jpd = row['jungle_prox_diff'] - row['m_jpd']
    
    temp_df = pd.DataFrame(row, index=[0])
    temp_df[["a_gd@8","a_cd@8","a_xpd@8","a_ka@8"]] = a_gd_8, a_cd_8, a_xpd_8, a_ka_8
    temp_df[["a_gd@14","a_cd@14","a_xpd@14","a_ka@14"]] = a_gd_14, a_cd_14, a_xpd_14, a_ka_14
    temp_df[['a_jp', 'a_jpd']] = a_jp, a_jpd
    
    adjusted_mid_stats = pd.concat([adjusted_mid_stats, temp_df])
adjusted_mid_stats = adjusted_mid_stats.reset_index(drop=True)

### SENDING DATA TO DB

In [4]:
from sqlalchemy import create_engine
import base64

#adjusted_mid_stats = adjusted_mid_stats.drop(columns=['Unnamed: 0'])
pg_password = 'U3ViYXJ1MjE2NQ=='
decrypted_pg_pwd = base64.b64decode(pg_password).decode('utf-8')
engine = create_engine('postgresql://postgres:%s@localhost:5432/postgres' % decrypted_pg_pwd)
adjusted_mid_stats.to_sql('midlaners', engine, index=False, if_exists='replace')

54